<!-- <p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Logo-gustave-roussy.jpg/1200px-Logo-gustave-roussy.jpg" alt="Logo 1" width="250"/>
  <img src="https://upload.wikimedia.org/wikipedia/en/thumb/3/3f/Qube_Research_%26_Technologies_Logo.svg/1200px-Qube_Research_%26_Technologies_Logo.svg.png" alt="Logo 2" width="200" style="margin-left: 20px;"/>
</p> -->

# Data Challenge : Leukemia Risk Prediction


*GOAL OF THE CHALLENGE and WHY IT IS IMPORTANT:*

The goal of the challenge is to **predict disease risk for patients with blood cancer**, in the context of specific subtypes of adult myeloid leukemias. The risk is measured through the **overall survival** of patients, i.e. the duration of survival from the diagnosis of the blood cancer to the time of death or last follow-up. The performance metric used in the challenge is the **IPCW-C-Index**.

*THE DATASETS*

The **training set is made of 3,323 patients**. The **test set is made of 1,193 patients**. For each patient, you have acces to CLINICAL data and MOLECULAR data. The details of the data are as follows:

- OUTCOME:
  * OS_YEARS = Overall survival time in years
  * OS_STATUS = 1 (death) , 0 (alive at the last follow-up)

- CLINICAL DATA, with one line per patient:
  
  * ID = unique identifier per patient
  * CENTER = clinical center
  * BM_BLAST = Bone marrow blasts in % (blasts are abnormal blood cells)
  * WBC = White Blood Cell count in Giga/L 
  * ANC = Absolute Neutrophil count in Giga/L
  * MONOCYTES = Monocyte count in Giga/L
  * HB = Hemoglobin in g/dL
  * PLT = Platelets coutn in Giga/L
  * CYTOGENETICS = A description of the karyotype observed in the blood cells of the patients, measured by a cytogeneticist. Cytogenetics is the science of chromosomes. A karyotype is performed from the blood tumoral cells. The convention for notation is ISCN (https://en.wikipedia.org/wiki/International_System_for_Human_Cytogenomic_Nomenclature). Cytogenetic notation are: https://en.wikipedia.org/wiki/Cytogenetic_notation. Note that a karyotype can be normal or abnornal. The notation 46,XX denotes a normal karyotype in females (23 pairs of chromosomes including 2 chromosomes X) and 46,XY in males (23 pairs of chromosomes inclusing 1 chromosme X and 1 chromsome Y). A common abnormality in the blood cancerous cells might be for exemple a loss of chromosome 7 (monosomy 7, or -7), which is typically asssociated with higher risk disease

In [ ]:
## TODO side note : mesures sont dans des échelles différentes donc attention à normaliser et standardiser les données

- GENE MOLECULAR DATA, with one line per patient per somatic mutation. Mutations are detected from the sequencing of the blood tumoral cells. 
We call somatic (= acquired) mutations the mutations that are found in the tumoral cells but not in other cells of the body.

  * ID = unique identifier per patient
  * CHR START END = position of the mutation on the human genome
  * REF ALT = reference and alternate (=mutant) nucleotide
  * GENE = the affected gene
  * PROTEIN_CHANGE = the consequence of the mutation on the protei that is expressed by a given gene
  * EFFECT = a broad categorization of the mutation consequences on a given gene.
  * VAF = Variant Allele Fraction = it represents the **proportion** of cells with the deleterious mutations. 

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored , concordance_index_ipcw
from sklearn.impute import SimpleImputer
from sksurv.util import Surv

import os
import re

In [ ]:
# Helper function for "now"
import datetime
def now_str():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d_%H-%M")

In [ ]:
# Files in the directory
# print(os.listdir("."))

# Clinical Data
# TODO : adapting to my local path
df = pd.read_csv("../data/X_train/clinical_train.csv")
df_eval = pd.read_csv("../data/X_test/clinical_test.csv")

# Molecular Data
maf_df = pd.read_csv("../data/X_train/molecular_train.csv")
maf_eval = pd.read_csv("../data/X_test/molecular_test.csv")

target_df = pd.read_csv("../data/target_train.csv")
# TODO: sera fourni le 15 ou le 17 mars
# target_df_test = pd.read_csv("./target_test.csv")

# Preview the data
df.head()

### Step 1: Data Preparation (clinical data only)

For survival analysis, we’ll format the dataset so that OS_YEARS represents the time variable and OS_STATUS represents the event indicator.

In [ ]:
### Starting of Data Cleaning and Preprocessing

# Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
# initial shape
# print(target_df.shape)
# drop rows with missing values
target_df.dropna(subset=['OS_YEARS', 'OS_STATUS'], inplace=True)
# final shape
# print(target_df.shape)
# percentage of rows dropped:
print(f'Percentage of initially dropt rows {(1 - target_df.shape[0] / df.shape[0]) * 100:.2f}%')

# Check the data types to ensure 'OS_STATUS' is boolean and 'OS_YEARS' is numeric
print(target_df[['OS_STATUS', 'OS_YEARS']].dtypes)

# Contarget_dfvert 'OS_YEARS' to numeric if it isn’t already
target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')

# Ensure 'OS_STATUS' is boolean
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

In [ ]:
### Feature Selection :

# Zero : features they selected for the Benchmark model :
features_basic = ['BM_BLAST', 'HB', 'PLT']
# TODO : Accuracies for each implemented model % set of features used, or if there is regularization or not...

# First: Naively add all the features  (except the Cytogenetics column):
# features= ['BM_BLAST', 'HB', 'PLT', 'WBC', 'ANC', 'MONOCYTES']
# Very naive : slight improvement

# Second : construct some features based on scientific knowledge, and add them to the dataframe and the features list
df['BLAST_per_WBC'] = df['BM_BLAST'] / (df['WBC'] + 1e-8)
df['ANC_per_WBC'] = df['ANC'] / (df['WBC'] + 1e-8)
# df['MONOCYTES_per_WBC'] = df['MONOCYTES'] / df['WBC']           # too much missing values, first drop it, then see how to exploit it
df["PL_per_HB"] = df['PLT'] / (df['HB'] + 1e-8)
# add these features to the features list
features_additional = ['BLAST_per_WBC', 'ANC_per_WBC', 'PL_per_HB']

df.head()

In [ ]:
# la meme pour le df_eval
df_eval['BLAST_per_WBC'] = df_eval['BM_BLAST'] / (df_eval['WBC'] + 1e-8)
df_eval['ANC_per_WBC'] = df_eval['ANC'] / (df_eval['WBC'] + 1e-8)
df_eval["PL_per_HB"] = df_eval['PLT'] / (df_eval['HB'] + 1e-8)
df_eval.head()

#### Step 1.B : Working on cytogenetic column data

In [ ]:
# Define patterns for key cytogenetic abnormalities
cytogenetic_markers = {
    "Has_Monosomy_7": r"-7",
    "Has_Trisomy_8": r"\+8",
    "Has_Del_5q": r"del\(5\)",
    "Has_Del_20q": r"del\(20\)",
    "Has_Del_17p": r"del\(17\)",
    "Has_T_3_3": r"t\(3;3\)",
    "Has_Complex_Karyotype": r"complex",
    "Has_Trisomy_21": r"\+21",
    "Has_Trisomy_22": r"\+22",
    "Has_T_9_11": r"t\(9;11\)",
    "Has_T_15_17": r"t\(15;17\)",
    "Has_Inv_3": r"inv\(3\)"
}
# TODO : quand les patterns ne correspondent pas exactement genre "del(5)" et "del(5q)"

# Function to extract cytogenetic abnormalities as separate binary features
def extract_cytogenetic_features(cytogenetics):
    features = {}
    
    # If missing or empty, set all to 0
    if pd.isna(cytogenetics) or cytogenetics.strip() == "":
        for key in cytogenetic_markers:
            features[key] = 0
        features["Complexity_Score"] = 0
        return features

    cytogenetics = cytogenetics.upper()
    
    # Check for each abnormality
    for key, pattern in cytogenetic_markers.items():
        features[key] = int(bool(re.search(pattern, cytogenetics)))

    # Compute complexity score (count of detected abnormalities)
    features["Complexity_Score"] = sum(features.values())

    return features

# Apply function to create new binary columns
cyto_features_df = df["CYTOGENETICS"].apply(extract_cytogenetic_features).apply(pd.Series)

# Merge new binary cytogenetic features into the main dataframe
df = pd.concat([df, cyto_features_df], axis=1)

# Drop the original cytogenetics column (no longer needed)
df.drop(columns=["CYTOGENETICS"], inplace=True)

# Display updated dataframe with binary cytogenetic features
df.head()

In [ ]:
# do the same for the eval set
cyto_features_df_eval = df_eval["CYTOGENETICS"].apply(extract_cytogenetic_features).apply(pd.Series)
df_eval = pd.concat([df_eval, cyto_features_df_eval], axis=1)
df_eval.drop(columns=["CYTOGENETICS"], inplace=True)
df_eval.head()

In [ ]:
features_cytogenetics = list(cyto_features_df.columns)
print("The cytogenetic data are :", features_cytogenetics)
print(len(features_cytogenetics))

In [ ]:
print(df_eval.shape)
print(df.shape)
# So we overall added 13 features to the dataframe

### Step 1.A : Process Molecular Data

In [ ]:
# Load molecular data
mutation_data = maf_df.copy()  # Assuming maf_df contains molecular variants

# One-hot encode gene mutations
mutation_features = mutation_data.pivot_table(index='ID', columns='GENE', values='VAF', aggfunc='max').fillna(0)

# One-hot encode mutation effects
mutation_effects = pd.get_dummies(mutation_data[['ID', 'EFFECT']], columns=['EFFECT']).groupby('ID').sum()

# Update feature list
molecular_features = list(mutation_features.columns)
molecular_effect = list(mutation_effects.columns)
# molecular_package = molecular_features + molecular_effect
# molecular_features = list(mutation_features.columns) + list(mutation_effects.columns)
print(df.shape)
print(mutation_effects.shape)
print(mutation_features.shape)

In [ ]:
# do the same to the eval set
mutation_data_eval = maf_eval.copy()
mutation_features_eval = mutation_data_eval.pivot_table(index='ID', columns='GENE', values='VAF', aggfunc='max').fillna(0)
mutation_effects_eval = pd.get_dummies(mutation_data_eval[['ID', 'EFFECT']], columns=['EFFECT']).groupby('ID').sum()

# define the list of molecular features of the eval set
molecular_features_eval = list(mutation_features_eval.columns)
molecular_effect_eval = list(mutation_effects_eval.columns)
# molecular_package_eval = molecular_features_eval + molecular_effect_eval
print(df_eval.shape)
print(mutation_effects_eval.shape)
print(mutation_features_eval.shape)

In [ ]:
# I want to ge the common molecular features between the train and the eval set
common_molecular_features = set(molecular_features).intersection(set(molecular_features_eval))
print("The number of common molecular feature is", len(common_molecular_features))

# # now I want to merge df to maf_df and df_eval to maf_evaln only on the common features
# print(list(common_molecular_features))
# print(mutation_features.columns)
df = df.merge(mutation_features[list(common_molecular_features)], on='ID', how='left').fillna(0)
df_eval = df_eval.merge(mutation_features_eval[list(common_molecular_features)], on='ID', how='left').fillna(0)

common_molecular_effect = set(molecular_effect).intersection(set(molecular_effect_eval))
print("The number of common molecular effect is",len(common_molecular_effect))
df = df.merge(mutation_effects[list(common_molecular_effect)], on='ID', how='left').fillna(0)
df_eval = df_eval.merge(mutation_effects_eval[list(common_molecular_effect)], on='ID', how='left').fillna(0)

In [ ]:
list_molecular_info = list(common_molecular_features) + list(common_molecular_effect)
features = features_basic + features_additional + features_cytogenetics + list_molecular_info
print(len(features))

In [ ]:
df.shape
df_eval.shape

In [ ]:
# TODO : est ce que any csv has been affected?

### Choose the features
TODO : need to implement feature selection

In [ ]:
# features = features_basic
# features = features_basic + features_additional
# features = features_basic + features_additional + features_cytogenetics

print("The features are:", features)
target = ['OS_YEARS', 'OS_STATUS']
print(len(features))

# print(len(molecular_features))
# TODO : faire comparaison entre le nombre de features et la taille de nos datasets

In [ ]:
# TODO : I can't take the common colums of the molecular features because they are not in the eval set (mais my split is deterministic.... comment remedier à ça?)

# features += molecular_features
# print("Updated feature set with molecular data:", features)
# print("Number of features:", len(features))   # 147 features ------- est ce que c'est abbérant % nombre de data points?

# # Take the list of common columns as the features
# features = list(common_columns)
# print(features)

In [ ]:
# Create the survival data format
X = df.loc[df['ID'].isin(target_df['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)

print(target_df[['OS_STATUS', 'OS_YEARS']].dtypes)
print(X.shape)

In [ ]:
# # TODO : FEATURE SELECTION 


# try the CoxPHFitter
# # Select top 50 best features
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_regression
# selector = SelectKBest(score_func=f_regression, k=50)
# X_selected = selector.fit_transform(X, y)

In [ ]:
# TODO : 0 quand on a pas les données -- c'est chaud 
# df.head()

### Step 2: Splitting the Dataset an,d Z norm
We’ll split the data into training and testing sets to evaluate the model’s performance.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Compute percentage of missing values per column
missing_percentage = X_train.isnull().mean() * 100
# Print the missing values percentage (genre de manière générale pas selon les catégories)
# print(missing_percentage)
# define a threshold for dropping columns with missing values
threshold = 10
# Print the columns with more than 10% missing values
# TODO : à voir in next runs
print(missing_percentage[missing_percentage > threshold])


# TODO : idk if we can do an imputation for the molecular data?????

# Survival-aware imputation for missing values
imputer = SimpleImputer(strategy="median")
X_train[features] = imputer.fit_transform(X_train[features])
X_test[features] = imputer.transform(X_test[features])

# TODO : est ce que faut faire chaque impuration pour chaque catégorie de features seules : genre features_cytogenetics, puis features_additional?

# Attention additional features have more missing values : 
# WBC           7.3 %
# ANC           4.6 %
# MONOCYTES    16.3 %

In [ ]:
# Z normalizing the data :
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
# We decide to keep the dataframe format
X_train_znorm = pd.DataFrame(data_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_znorm = pd.DataFrame(data_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

# DONC : 
# X_train est un dataframe, NON Z normed
# X_train_znorm est un dataframe, Z normed

# TODO : est ce quil faut Znorm the y_train?

In [ ]:
# TODO : maybe drop les datas ou ya bcp de manque??

# features.remove('PL_per_HB')

### Step 3: Training Standard Machine Learning Methods

In this step, we train a standard LightGBM model on survival data, but we do not account for censoring. Instead of treating the event status, we use only the observed survival times as the target variable. This approach disregards whether an individual’s event (e.g., death) was observed or censored, effectively treating the problem as a standard regression task. While this method provides a basic benchmark, it may be less accurate than survival-specific models (but still be explored!), as it does not leverage the information contained in censored observations.

In [ ]:
# Import necessary libraries
import lightgbm as lgb
import xgboost as xgb
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

# Define LightGBM parameters
lgbm_params = {
    'max_depth': 3,         # TODO : when adding features try to increase the depth of the tree
    'learning_rate': 0.05,
    'verbose': -1,
    # 'alpha': 100,
}

# Prepare the data for LightGBM
# Scale the target (OS_YEARS) to reduce skew, apply weights based on event status
X_train_lgb = X_train_znorm  # Features for training
y_train_transformed = y_train['OS_YEARS']

# Create LightGBM dataset
train_dataset = lgb.Dataset(X_train_lgb, label=y_train_transformed)

# Train the LightGBM model
model = lgb.train(params=lgbm_params, train_set=train_dataset)

# Make predictions on the training and testing sets
pred_train = -model.predict(X_train_znorm)
pred_test = -model.predict(X_test_znorm)

# Evaluate the model using Concordance Index IPCW
train_ci_ipcw = concordance_index_ipcw(y_train, y_train, pred_train, tau=7)[0]
test_ci_ipcw = concordance_index_ipcw(y_train, y_test, pred_test, tau=7)[0]
print(f"LightGBM Survival Model Concordance Index IPCW on train: {train_ci_ipcw:.2f}")
print(f"LightGBM Survival Model Concordance Index IPCW on test: {test_ci_ipcw:.2f}")


#### Training a standard XGBoost model

In [ ]:
# train a XGBoost model for survival analysis
xgb_params = {
    # "objective": "rank:pairwise",  # Optimized for ranking risk scores
    # "eval_metric": "ndcg",  # Normalized Discounted Cumulative Gain
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0,
}

# Prepare the data for XGBoost
X_train_xgb = X_train_znorm
y_train_xgb = y_train['OS_YEARS']

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_xgb, label=y_train_xgb)

# Train the XGBoost model
xgb_model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=100)

# Make predictions on the training and testing sets
dtest_train = xgb.DMatrix(X_train_znorm)
dtest_test = xgb.DMatrix(X_test_znorm)
pred_train_xgb = -xgb_model.predict(dtest_train)
pred_test_xgb = -xgb_model.predict(dtest_test)

# Evaluate the model using Concordance Index IPCW
train_ci_ipcw_xgb = concordance_index_ipcw(y_train, y_train, pred_train_xgb, tau=7)[0]
test_ci_ipcw_xgb = concordance_index_ipcw(y_train, y_test, pred_test_xgb, tau=7)[0]
print(f"XGBoost Survival Model Concordance Index IPCW on train: {train_ci_ipcw_xgb:.2f}")
print(f"XGBoost Survival Model Concordance Index IPCW on test: {test_ci_ipcw_xgb:.2f}")

#### Grid Search for the XGBoost 

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'verbosity': [0]
}

# Initialize the XGBoost model
xgb_model = xgb.XGBRegressor()

# Initialize GridSearchCV with the XGBoost model and parameter grid
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
# TODO : il fait un grid search avec un negative mean squared error

# Prepare the data for XGBoost
X_train_xgb = X_train
y_train_xgb = y_train['OS_YEARS']

# Fit the grid search to the training data
grid_search.fit(X_train_xgb, y_train_xgb)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters found: {best_params}")
print(f"Best score found: {best_score}")

# Train the XGBoost model with the best parameters
best_xgb_model = xgb.XGBRegressor(**best_params)
best_xgb_model.fit(X_train_xgb, y_train_xgb)

# Make predictions on the training and testing sets
dtest_train = xgb.DMatrix(X_train)
dtest_test = xgb.DMatrix(X_test)
pred_train_xgb = -best_xgb_model.predict(dtest_train)
pred_test_xgb = -best_xgb_model.predict(dtest_test)

# Evaluate the model using Concordance Index IPCW
train_ci_ipcw_xgb = concordance_index_ipcw(y_train, y_train, pred_train_xgb, tau=7)[0]
test_ci_ipcw_xgb = concordance_index_ipcw(y_train, y_test, pred_test_xgb, tau=7)[0]
print(f"XGBoost Survival Model Concordance Index IPCW on train: {train_ci_ipcw_xgb:.2f}")
print(f"XGBoost Survival Model Concordance Index IPCW on test: {test_ci_ipcw_xgb:.2f}")


In [ ]:
# df.head()

In [ ]:
# TODO : save submission file of the lgb, and xgboost methods
# TODO : train light gbm with the number of mutations

**Observation for LightGBM** 

- le modèle commence à overfitter sur le train set quand on ajoute des features (ratio des PL et HB et ratios similaires, et aussi cytogenetic data)
- il a encore plus overfitté sur le train avec l'ajout des data moléculaires

In [ ]:
# VISUALIZING THE 
# 
# # Assuming the LightGBM model is defined as `model`
# plt.figure(figsize=(20, 10))
# lgb.plot_tree(model, tree_index=0, figsize=(20, 10), show_info=['split_gain', 'internal_value', 'internal_count', 'leaf_count'])
# plt.title("First Tree in LightGBM Model")
# plt.show()

### Step 4: Cox Proportional Hazards Model

To account for censoring in survival analysis, we use a Cox Proportional Hazards (Cox PH) model, a widely used method that estimates the effect of covariates on survival times without assuming a specific baseline survival distribution. The Cox PH model is based on the hazard function, $h(t | X)$, which represents the instantaneous risk of an event (e.g., death) at time $t$ given covariates $X$. The model assumes that the hazard can be expressed as:

$$h(t | X) = h_0(t) \exp(\beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p)$$


where $h_0(t)$ is the baseline hazard function, and $\beta$ values are coefficients for each covariate, representing the effect of $X$ on the hazard. Importantly, the proportional hazards assumption implies that the hazard ratios between individuals are constant over time. This approach effectively leverages both observed and censored survival times, making it a more suitable method for survival data compared to standard regression techniques that ignore censoring.


In [ ]:
# Initialize and train the Cox Proportional Hazards model
cox = CoxPHSurvivalAnalysis(alpha=10.0)
# TODO : ajuster les hyperparamètres
cox.fit(X_train, y_train)

# Evaluate the model using Concordance Index IPCW
cox_cindex_train = concordance_index_ipcw(y_train, y_train, cox.predict(X_train), tau=7)[0]
cox_cindex_test = concordance_index_ipcw(y_train, y_test, cox.predict(X_test), tau=7)[0]
print(f"Cox Proportional Hazard Model Concordance Index IPCW on train: {cox_cindex_train:.2f}")
print(f"Cox Proportional Hazard Model Concordance Index IPCW on test: {cox_cindex_test:.2f}")

In [ ]:
# # Initialize and train the Cox Proportional Hazards model
# from lifelines import CoxPHFitter
# cox2 = CoxPHFitter()
# # TODO : ajuster les hyperparamètres
# cox2.fit(df, duration_col='OS_YEARS', event_col='OS_STATUS')
# # fit(X_train, y_train)

# # Evaluate the model using Concordance Index IPCW
# cox2_cindex_train = concordance_index_ipcw(y_train, y_train, cox2.predict(X_train), tau=7)[0]
# cox2_cindex_test = concordance_index_ipcw(y_train, y_test, cox2.predict(X_test), tau=7)[0]
# print(f"Cox Proportional Hazard Model Concordance Index IPCW on train: {cox2_cindex_train:.2f}")
# print(f"Cox Proportional Hazard Model Concordance Index IPCW on test: {cox2_cindex_test:.2f}")

In [ ]:
# DEBUGGING suggested by chatgpt quand on prend en compte les mutations data :

import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = X_train.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", fmt=".2f")
plt.title("Feature Correlation Matrix")
# plt.show()

# TODO : ya bcp de correlation entre certains features!!!!! dans le petit carré en haut à gauche ----- faire de la feature selection


In [ ]:
# verify if there are totally null columns
null_columns = X_train.columns[X_train.isnull().all()]
print(null_columns)

# TODO : feature selection
# TODO : expand le recouvrement dans le cas de cytogenetic data

In [ ]:
# define a threshold of correlation and print all the pairs of features that are highly correlated
threshold = 0.7
print(len(features))
for i in range(len(features)):
    for j in range(i+1, len(features)):
        if corr_matrix.iloc[i, j] > threshold:
            print(f"Features {features[i]} and {features[j]} are highly correlated with a correlation of {corr_matrix.iloc[i, j]:.2f}")

# print("3rd and sixth features are highly correlated")
# print(corr_matrix.iloc[2, 5])
# print("those features are :")
# print(features[2],"and", features[5])

In [ ]:
# now that we have the pairs of highly correlated features, we can drop one of them
# Drop the highly correlated features

# TODO : attention fix which feature we keep or remove
corr_feature = 'PL_per_HB'
features.remove(corr_feature)
# features.remove('PLT')

# update the training and testing sets
X_train = X_train.drop(columns=[corr_feature])
X_test = X_test.drop(columns=[corr_feature])

X = X.drop(columns=[corr_feature])

print(len(features))


# recompute the correlation matrix
corr_matrix = X_train.corr()

In [ ]:
# plt.figure(figsize=(12, 8))
# sns.heatmap(corr_matrix, annot=False, cmap="coolwarm", fmt=".2f")
# plt.title("Feature Correlation Matrix")

threshold = 0.7
print(len(features))
for i in range(len(features)):
    for j in range(i+1, len(features)):
        if corr_matrix.iloc[i, j] > threshold:
            print(f"Features {features[i]} and {features[j]} are highly correlated with a correlation of {corr_matrix.iloc[i, j]:.2f}")

### Step 5: Naive Approach to Incorporate Mutations

In this step, we take a very naive approach to account for genetic mutations by simply counting the total number of somatic mutations per patient. Instead of analyzing specific mutations or their biological impact, we use this aggregate count as a basic feature to reflect the mutational burden for each individual. Although simplistic, this feature can serve as a general indicator of genetic variability across patients, which may influence survival outcomes. More sophisticated mutation analysis could be incorporated in future models to improve predictive power.


In [ ]:
# Step: Extract the number of somatic mutations per patient
# Group by 'ID' and count the number of mutations (rows) per patient
tmp = maf_df.groupby('ID').size().reset_index(name='Nmut')

# Merge with the training dataset and replace missing values in 'Nmut' with 0
df_2 = df.merge(tmp, on='ID', how='left').fillna({'Nmut': 0})

In [ ]:
# do the same for the df_eval :
# Adding the Nmut feature to the evaluation dataset
tmp_eval = maf_eval.groupby('ID').size().reset_index(name='Nmut')

# Merge with the training dataset and replace missing values in 'Nmut' with 0
df_eval = df_eval.merge(tmp_eval, on='ID', how='left').fillna({'Nmut': 0})

df_eval.head()

In [ ]:
# Select features
# features = ['BM_BLAST', 'HB', 'PLT', 'Nmut']
# features = ['BM_BLAST', 'HB', 'PLT', 'WBC', 'ANC', 'MONOCYTES', 'Nmut']
# features = features_basic + ['BLAST_per_WBC', 'ANC_per_WBC', 'PL_per_HB', 'Nmut']
# features = features_basic + ['Nmut']
features = features + ['Nmut']
target = ['OS_YEARS', 'OS_STATUS']

# Create the survival data format
X = df_2.loc[df_2['ID'].isin(target_df['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# TODO : change the random state?

In [ ]:
# Survival-aware imputation for missing values
imputer = SimpleImputer(strategy="median")
X_train[features] = imputer.fit_transform(X_train[features])
X_test[features] = imputer.transform(X_test[features])

In [ ]:
print("the features are:", features)
print("the number of features:", len(features))

In [ ]:
# Z normalizing the data
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
X_train_znorm = pd.DataFrame(data_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_znorm = pd.DataFrame(data_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [ ]:
# Initialize and train the Cox Proportional Hazards model
# cox = CoxPHSurvivalAnalysis()
cox = CoxPHSurvivalAnalysis(alpha=10.)
# TODO : ajuster les hyperparamètres
cox.fit(X_train, y_train)
# observation : better results without Z normalization

# Evaluate the model using Concordance Index IPCW
cox_cindex_train = concordance_index_ipcw(y_train, y_train, cox.predict(X_train), tau=7)[0]
cox_cindex_test = concordance_index_ipcw(y_train, y_test, cox.predict(X_test), tau=7)[0]
print(f"Cox Proportional Hazard Model Concordance Index IPCW on train: {cox_cindex_train:.2f}")
print(f"Cox Proportional Hazard Model Concordance Index IPCW on test: {cox_cindex_test:.2f}")

### Step 6 : Random Survival Forest model 

In [ ]:
# # Selecting the adequate features for the Random Survival Forest model
# features = features_basic + features_additional + features_cytogenetics + ['Nmut']
# or just using all the features

# # Survival-aware imputation for missing values
# imputer = SimpleImputer(strategy="median")
# X_train[features] = imputer.fit_transform(X_train[features])
# X_test[features] = imputer.transform(X_test[features])

In [ ]:
# Initialize and train Random Survival Forest
rsf = RandomSurvivalForest(n_estimators=200, min_samples_split=10, min_samples_leaf=5, max_depth=5, random_state=42, oob_score=True)
# de base ken n_estimators=200, tried avec 300 too
# TODO : what does the oob score do?
rsf.fit(X_train_znorm, y_train)
# Résultat pareil, qu'on Z- normalise ou pas

# Evaluate using IPCW Concordance Index
rsf_cindex_train = concordance_index_ipcw(y_train, y_train, rsf.predict(X_train_znorm), tau=7)[0]
rsf_cindex_test = concordance_index_ipcw(y_train, y_test, rsf.predict(X_test_znorm), tau=7)[0]

print(f"RSF Concordance Index IPCW on train: {rsf_cindex_train:.2f}")
print(f"RSF Concordance Index IPCW on test: {rsf_cindex_test:.2f}")

In [ ]:
# Training for a different horizon : TAU = 5 years
# -------------- okay done
###### NEXT :
# Train on "the best parameters" for the Random Survival Forest model (found with GridSearchCV)

# Initialize and train Random Survival Forest
rsf2 = RandomSurvivalForest(n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_depth=3, random_state=42)
rsf2.fit(X_train, y_train)

# Evaluate using IPCW Concordance Index
rsf2_cindex_train = concordance_index_ipcw(y_train, y_train, rsf2.predict(X_train), tau=7)[0]
rsf2_cindex_test = concordance_index_ipcw(y_train, y_test, rsf2.predict(X_test), tau=7)[0]

print(f"RSF Concordance Index IPCW on train: {rsf2_cindex_train:.2f}")
print(f"RSF Concordance Index IPCW on test: {rsf2_cindex_test:.2f}")

In [ ]:
# TODO : regularize the RSF model

In [ ]:
# # Best parameters found: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
# # Initialize and train Random Survival Forest
# wow = RandomSurvivalForest(n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_depth=3, random_state=42)
# wow.fit(X_train, y_train)

# # Evaluate using IPCW Concordance Index
# wow_cindex_train = concordance_index_ipcw(y_train, y_train, wow.predict(X_train), tau=7)[0]
# wow_cindex_test = concordance_index_ipcw(y_train, y_test, wow.predict(X_test), tau=7)[0]

# print(f"RSF Concordance Index IPCW on train: {wow_cindex_train:.2f}")
# print(f"RSF Concordance Index IPCW on test: {wow_cindex_test:.2f}")

In [ ]:
print(len(features))
# TODO : maybe mix long and short term predictions?

### Cross Validation for RSF model

In [ ]:
from sklearn.model_selection import cross_val_score
from sksurv.metrics import concordance_index_censored

# Define a function to compute the IPCW concordance index for cross-validation
def ipcw_cindex(estimator, X, y):
    predictions = estimator.predict(X)
    return concordance_index_ipcw(y, y, predictions, tau=7)[0]

# Perform cross-validation
cv_scores = cross_val_score(rsf, X, y, cv=5, scoring=ipcw_cindex)

# Print the cross-validation scores
print(f"Cross-validation IPCW C-index scores: {cv_scores}")
print(f"Mean IPCW C-index: {cv_scores.mean():.2f}")
print(f"Standard Deviation of IPCW C-index: {cv_scores.std():.2f}")

In [ ]:
# Cross Validation for the RSF2 model (the one with "best parameters")
cv_scores2 = cross_val_score(rsf2, X, y, cv=5, scoring=ipcw_cindex)

# Print the cross-validation scores
print(f"Cross-validation IPCW C-index scores: {cv_scores2}")
print(f"Mean IPCW C-index: {cv_scores2.mean():.2f}")
print(f"Standard Deviation of IPCW C-index: {cv_scores2.std():.2f}")

In [ ]:
# Grid search for the best hyperparameters for RSF
from sklearn.model_selection import GridSearchCV
# Define the parameter grid for the Random Survival Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5]
}

# Initialize the Random Survival Forest model
rsf = RandomSurvivalForest(random_state=42)

# Initialize GridSearchCV with the RSF model and parameter grid
grid_search = GridSearchCV(estimator=rsf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters found: {best_params}")
print(f"Best score found: {best_score}")

# TODO : faire un grid search pour toutes les méthodes


*****************
### Inference on test set

In [ ]:
# # Adding the Nmut feature to the evaluation dataset
# tmp_eval = maf_eval.groupby('ID').size().reset_index(name='Nmut')

# # Merge with the training dataset and replace missing values in 'Nmut' with 0
# df_eval = df_eval.merge(tmp_eval, on='ID', how='left').fillna({'Nmut': 0})

df_eval.head()

In [ ]:
# df_eval.drop(columns="Nmut_x", inplace=True)    
# df_eval.rename(columns={"Nmut_y": "Nmut"}, inplace=True)
# df_eval.head()

In [ ]:
# Display the features we have selected
# print(features)
# df_eval.head()
df_eval[features] = imputer.transform(df_eval[features])
X_eval_znorm = pd.DataFrame(data_scaler.transform(df_eval[features]), columns=features, index=df_eval.index)

# cox_prediction_on_test_set = cox.predict(df_eval.loc[:, features])

In [ ]:
# df_eval[features] = imputer.transform(df_eval[features])
rsf_prediction_on_test_set = rsf.predict(X_eval_znorm.loc[:, features])
# lgb_prediction_on_test_set = model.predict(df_eval.loc[:, features])

#### Saving the submission

In [ ]:
# submission = pd.Series(prediction_on_test_set, index=df_eval['ID'], name='OS_YEARS')
# cox_test_submission = pd.Series(cox_prediction_on_test_set, index=df_eval['ID'], name='risk_score')
rsf_test_submission = pd.Series(rsf_prediction_on_test_set, index=df_eval['ID'], name='risk_score')

In [ ]:
rsf_test_submission

In [ ]:
os.makedirs('./output', exist_ok=True)

# # I just want the date and hour and minute
# import datetime
# now = datetime.datetime.now()
# now = now.strftime("%Y-%m-%d_%H-%M")

# submission.to_csv(f'./output/submission_{now}_rsf_ok.csv')
# cox_test_submission.to_csv(f'./output/submission_{now_str()}_cox.csv')
rsf_test_submission.to_csv(f'./output/submission_{now_str()}_rsf.csv')
# submission.to_csv('./output/all_clinical_features_submission.csv')

***************
### Stacking a meta model on COX + RSF
First, trying with a linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

# Load test survival data
# rsf_preds = pd.read_csv("./output/submission_2025-03-10_00-29_rsf.csv", index_col="ID")  
# cox_preds = pd.read_csv("./output/submission_2025-03-10_00-29_cox.csv", index_col="ID")  

# Z-normalize the survival scores obtain by methods RSF and COX, using StandardScaler
dd_scaler = StandardScaler()

# TODO : Attention : each model has been trained on Znormed ou pas ?!

# Train set :
rsf_pred_train = dd_scaler.fit_transform(rsf.predict(X_train_znorm).reshape(-1, 1)).flatten()
cox_pred_train = dd_scaler.fit_transform(cox.predict(X_train).reshape(-1, 1)).flatten()

# Test set :
rsf_pred_test = dd_scaler.transform(rsf.predict(X_test_znorm).reshape(-1, 1)).flatten()
cox_pred_test = dd_scaler.transform(cox.predict(X_test).reshape(-1, 1)).flatten()

# rsf_pred_test = (rsf_pred_test - rsf_pred_test.mean()) / rsf_pred_test.std()
# cox_pred_test = (cox_pred_test - cox_pred_test.mean()) / cox_pred_test.std()


# Stack RSF & Cox risk scores
stacked_X_train = pd.DataFrame({"rsf_score": rsf_pred_train, "cox_score": cox_pred_train}, index=X_train.index)
stacked_X_test = pd.DataFrame({"rsf_score": rsf_pred_test, "cox_score": cox_pred_test}, index=X_test.index)

# Train a meta-learner : logistic regression
meta_model = LinearRegression()
# TODO : finetuner le LR de la logistic regression par exemple
meta_model.fit(stacked_X_train, y_train["OS_YEARS"])

# Predict combined survival scores
# predict or predict_proba ????
ensemble_preds = -meta_model.predict(stacked_X_test)
train_ensemble_preds = -meta_model.predict(stacked_X_train)

# print(f"y_train shape: {y_train.shape}")
# print(f"y_test shape: {y_test.shape}")
# print(f"ensemble_preds shape: {ensemble_preds.shape}")

# Evaluate IPCW C-index
train_ensemble_cindex = concordance_index_ipcw(y_train, y_train, train_ensemble_preds, tau=7)[0]
ensemble_cindex = concordance_index_ipcw(y_train, y_test, ensemble_preds, tau=7)[0]
print(f"Linear Meta-Learner IPCW C-index on train: {train_ensemble_cindex:.2f}")
print(f"Linear Meta-Learner IPCW C-index: {ensemble_cindex}")

In [ ]:
print(y_train.shape)
# print(y_train[:,0])

In [ ]:
# did a Z norm maybe before applying the meta model Logistic regression 
# TODO : feature selection

#### XGBoost as meta-learner on COX + RSF

In [ ]:
import xgboost as xgb
from sksurv.metrics import concordance_index_ipcw

# Stack RSF & Cox risk scores
# non Z norm
# stacked_X_train = pd.DataFrame({"rsf_score": rsf.predict(X_train), "cox_score": cox.predict(X_train)}, index=X_train.index)
# stacked_X_test = pd.DataFrame({"rsf_score": rsf.predict(X_test), "cox_score": cox.predict(X_test)}, index=X_test.index)
# Z norm
stacked_X_train = pd.DataFrame({"rsf_score": rsf_pred_train, "cox_score": cox_pred_train}, index=X_train.index)
stacked_X_test = pd.DataFrame({"rsf_score": rsf_pred_test, "cox_score": cox_pred_test}, index=X_test.index)

# Convert survival labels to a format usable by XGBoost (duration and event)
y_train_event = y_train["OS_STATUS"].astype(int)  # 1 if event occurred, 0 otherwise
y_train_time = y_train["OS_YEARS"]  # Survival time

# Train an XGBoost model for survival ranking
dtrain = xgb.DMatrix(stacked_X_train, label=y_train_time)  # Use time as label for ranking
dtest = xgb.DMatrix(stacked_X_test)

params = {
    "objective": "survival:cox",  # Optimized for ranking risk scores
    "eval_metric": "aft-nloglik",  # Normalized Discounted Cumulative Gain
    "learning_rate": 0.0001,  # Learning rate
    "max_depth": 4,  # Small depth to avoid overfitting
    "subsample": 0.7,  # Use 80% of data per tree
    "colsample_bytree": 0.7,  # Use 80% of features per tree
    "lambda": 0.01,  # L2 regularization
    "alpha": 0.1,  # L1 regularization
}

xgb_model = xgb.train(params, dtrain, num_boost_round=200)

# Predict survival risk scores
train_ensemble_preds = xgb_model.predict(dtrain)
ensemble_preds = xgb_model.predict(dtest)

# Evaluate IPCW C-index
# train ipcw c-index
ensemble_cindex_train = concordance_index_ipcw(y_train, y_train, train_ensemble_preds, tau=7)[0]
# test ipcw c-index
ensemble_cindex = concordance_index_ipcw(y_train, y_test, ensemble_preds, tau=7)[0]
print(f"Optimized Meta-Learner IPCW C-index on train: {ensemble_cindex_train}")
print(f"Optimized Meta-Learner IPCW C-index on test : {ensemble_cindex}")

In [ ]:
print(stacked_X_train.corr())

In [ ]:
# Inference time

# Load test survival data
# rsf_preds = pd.read_csv("./output/submission_2025-03-10_00-29_rsf.csv", index_col="ID")
# cox_preds = pd.read_csv("./output/submission_2025-03-10_00-29_cox.csv", index_col="ID")  

rsf_pred_eval = rsf.predict(X_eval_znorm)
cox_pred_eval = cox.predict(df_eval[features])
rsf_predd = dd_scaler.transform(rsf_pred_eval.reshape(-1, 1)).flatten()
cox_predd = dd_scaler.transform(cox_pred_eval.reshape(-1, 1)).flatten()

# print(rsf_predd)
 
# Stack RSF & Cox risk scores of the test set
stacked_inference = pd.DataFrame({"rsf_score": rsf_predd, "cox_score": cox_predd}, index=df_eval.index)
# dinference = xgb.DMatrix(stacked_inference)

# Predict survival risk scores
# inference_preds = xgb_model.predict(dinference)
inference_preds = - meta_model.predict(stacked_inference)

# Save the submission file
submission = pd.Series(inference_preds, index=df_eval['ID'], name='risk_score')
submission
submission.to_csv(f'./output/submission_meta(cox_rsf)_xgboost_{now_str()}_ensemble.csv')

*******************
### Step 7 : try combining the 2 models' (COX, RSF) results by hand :
genre faire des combinaisons


In [ ]:
# os.listdir('./output')
# TODO : test chaque methode seule deja sur la platforme
# - tested les resultats de RSF, turns out : it doesn't need to be formatted before


# side note for conversion:
# # Convert Cox risk scores to survival time estimates
# cox_preds["survival_time"] = np.exp(-cox_preds["risk_score"])  # Transforming risk scores
# # print(cox_preds.head())


# # try to combine, COX and RSF model : classic linear averaging ------------- doesn't work well
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
# TRYING A NEW STRATEGY TO COMBINE BOTH RESULTS :

os.makedirs('./output', exist_ok=True)

# Load RSF and Cox predictions
rsf_preds = pd.read_csv("./output/submission_2025-03-11_19-17_rsf.csv", index_col="ID")  
cox_preds = pd.read_csv("./output/submission_2025-03-11_19-17_cox.csv", index_col="ID") 

# Standardize RSF and Cox risk scores
rsf_preds["z_score"] = (rsf_preds["risk_score"] - rsf_preds["risk_score"].mean()) / rsf_preds["risk_score"].std()
cox_preds["z_score"] = (cox_preds["risk_score"] - cox_preds["risk_score"].mean()) / cox_preds["risk_score"].std()

# Define weights based on performance
w_rsf = 0.8     # 0.745 / (0.745 + 0.72)  # Weight proportional to C-index
w_cox = 0.2     # 0.72 / (0.745 + 0.72)
# tried with weights (0.8, 0.2) but didn't save or submit the results

# Weighted combination using z-scores
combined_risk_score = (w_rsf * rsf_preds["z_score"] + w_cox * cox_preds["z_score"])

# Store final risk scores
final_preds = pd.DataFrame({"ID": rsf_preds.index, "risk_score": combined_risk_score})
# final_preds.to_csv(f"./output/combined_risk_scores_zscore_{now_str()}.csv", index=False)

In [ ]:
# final_preds

***************
### Step 8 : Trying a DeepLearning model

In [ ]:
from pycox.models import CoxPH
import torch
from torchtuples.practical import MLPVanilla
from sklearn.preprocessing import StandardScaler


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preparing the data
# we have already done the Z normalization : X_train_znorm, X_test_znorm
# X_train_znorm est un df et X_train_t est un tensor

# Convert data to tensors
X_train_t = torch.tensor(X_train_znorm.values, dtype=torch.float32).to(device)
X_test_t = torch.tensor(X_test_znorm.values, dtype=torch.float32).to(device)

# saving the original y_train de coté
y_train_org = y_train.copy()

# Ensure target columns are the correct type
y_train["OS_STATUS"] = y_train["OS_STATUS"].astype(int)  # Ensure it's an integer
y_train["OS_YEARS"] = y_train["OS_YEARS"].astype(float)  # Ensure it's a float

# Convert target variables into PyTorch tensors (Manual conversion)
y_train_t = (
    torch.tensor(y_train["OS_YEARS"].copy(), dtype=torch.float64).to(device),  # Time to event
    torch.tensor(y_train["OS_STATUS"].copy(), dtype=torch.int64).to(device)    # Event occurred (1=event, 0=censored)
)

#### First implementation sanity check

In [ ]:
# Define a simple MLP model
net = MLPVanilla(
    in_features=X_train_znorm.shape[1], num_nodes=[64, 32], out_features=1, 
    batch_norm=True, dropout=0.2, output_bias=False     # dropout =0.4
).to(device)
# Finetune dropout to prevent overfitting

# Train DeepSurv
model = CoxPH(net)
model.fit(X_train_t, y_train_t, batch_size=128, epochs=300, verbose=False)
# TOD : tune the epoch nbr

# Predict risk scores
deep_risk_scores = model.predict(X_test_t).cpu().detach().numpy()   # Predict risk scores (reverse sign)
deep_risk_scores = deep_risk_scores.reshape(-1)


# Evaluate
# print(len(y_train_t))
# print(y_train.shape)
# print(y_test.shape)
# print(deep_risk_scores.shape)
deep_cindex = concordance_index_ipcw(y_train, y_test, deep_risk_scores, tau=7)[0]
print(f"DeepSurv IPCW C-index: {deep_cindex}")


#### Grid Search for the DeepSurv approach

In [ ]:
from sklearn.model_selection import ParameterGrid

param_grid = {
    'num_nodes': [[64, 32], [128, 64, 32], [256, 128, 64]],
    'dropout': [0.2, 0.3, 0.4],
    'lr': [1e-3, 5e-4, 1e-4]
}

best_cindex = 0
best_params = None

for params in ParameterGrid(param_grid):
    net = MLPVanilla(
        in_features=X_train.shape[1], 
        num_nodes=params['num_nodes'], 
        out_features=1, 
        batch_norm=True, 
        dropout=params['dropout'], 
        output_bias=False
    ).to(device)

    model = CoxPH(net, optimizer=torch.optim.Adam)

    # Set optimizer learning rate
    model.optimizer.set_lr(params['lr']) 

    model.fit(X_train_t, y_train_t, batch_size=128, epochs=300, verbose=False)

    deep_risk_scores = model.predict(X_test_t).cpu().detach().numpy().reshape(-1)
    cindex = concordance_index_censored(y_test["OS_STATUS"], y_test["OS_YEARS"], deep_risk_scores)[0]
    
    print(f"Params: {params} | C-index: {cindex}")

    # Track best parameters
    if cindex > best_cindex:
        best_cindex = cindex
        best_params = params

print(f"Best parameters: {best_params} | Best C-index: {best_cindex}")

# TODO : print both the train and test c-index
# TODO : evaluate on the same metric as the other models.....


#### Our optimal params model

In [ ]:
# Test on optimal parameters obtained by grid search :
# Best parameters: {'dropout': 0.4, 'lr': 0.0001, 'num_nodes': [64, 32]} | Best C-index: 0.7280682581068211

# Define a simple MLP model
net = MLPVanilla(
        in_features=X_train_znorm.shape[1], 
        num_nodes= [64, 32, 16],
        out_features=1, 
        batch_norm=True, 
        dropout= 0.2,   #0.4
        output_bias=False
    ).to(device)
# Increasing dropout prevents overfitting

# Train DeepSurv
deepsurv = CoxPH(net, optimizer=torch.optim.Adam)
# Set optimizer learning rate
deepsurv.optimizer.set_lr(0.0001)  # 0.0001
deepsurv.optimizer.weight_decay = 1e-4  # L2 regularization
# TODO : Strong overfitting on the train data

deepsurv.fit(X_train_t, y_train_t, batch_size=128, epochs=200, verbose=False)
# TODO : tuning the epoch nbr?
# TODO : est ce que on peut plot la loss au fur et à mesure? 

train_deep_risk_scores = deepsurv.predict(X_train_t).cpu().detach().numpy().reshape(-1)
test_deep_risk_scores = deepsurv.predict(X_test_t).cpu().detach().numpy().reshape(-1)

# # compute train and test c-index
# deep_train_cindex = concordance_index_censored(y_train["OS_STATUS"], y_train["OS_YEARS"], train_deep_risk_scores)[0]
# deep_test_cindex = concordance_index_censored(y_test["OS_STATUS"], y_test["OS_YEARS"], test_deep_risk_scores)[0]
# Compute the train and test ipcw c-index
deep_train_cindex = concordance_index_ipcw(y_train, y_train, train_deep_risk_scores, tau=7)[0]
deep_test_cindex = concordance_index_ipcw(y_train, y_test, test_deep_risk_scores, tau=7)[0]
print(f"DeepSurv IPCW C-index on train: {deep_train_cindex}")
print(f"DeepSurv IPCW C-index on test: {deep_test_cindex}")

In [ ]:
# TODO : do we upload the submissions of XGBoost (wahad men li abel bess ayyeh?)

************
### Training strategy to be able to stack model :

In [ ]:
### The one RSF :
# Initialize the Random Survival Forest
rsf_stack = RandomSurvivalForest(n_estimators=200, min_samples_split=10, min_samples_leaf=5, max_depth=5, random_state=42, oob_score=True)

# Initialize the DeepSurv model :
net_stack = MLPVanilla(
        in_features=X_train_znorm.shape[1], 
        num_nodes= [64, 32],
        out_features=1, 
        batch_norm=True, 
        dropout= 0.2,   
        output_bias=False
    ).to(device)
# Increasing dropout prevents overfitting

# Train DeepSurv
deepsurv_stack = CoxPH(net_stack, optimizer=torch.optim.Adam)
# Set optimizer learning rate
deepsurv_stack.optimizer.set_lr(0.0001)  # 0.0001
deepsurv_stack.optimizer.weight_decay = 1e-4  # L2 regularization
# TODO : Strong overfitting on the train data

In [ ]:
# # Z normalizing the data :
# from sklearn.preprocessing import StandardScaler
# data_scaler = StandardScaler()
# # We decide to keep the dataframe format
# X_train_znorm = pd.DataFrame(data_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
# X_test_znorm = pd.DataFrame(data_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Store out-of-fold predictions
train_rsf_oof = np.zeros(len(X_train_znorm))
train_deepsurv_oof = np.zeros(len(X_train_znorm))
# print(train_rsf_oof.shape)
# print(train_deepsurv_oof.shape)

y_train_org = y_train.copy()
# TODO : comment je gere ma data....

# Ensure target columns are the correct type
y_train["OS_STATUS"] = y_train["OS_STATUS"].astype(int)  # Ensure it's an integer
y_train["OS_YEARS"] = y_train["OS_YEARS"].astype(float)  # Ensure it's a float


for train_idx, val_idx in kf.split(X_train_znorm):
    # selected data :
    X_train_selected = X_train_znorm.iloc[train_idx]
    X_val_selected = X_train_znorm.iloc[val_idx]

    # Train RSF & DeepSurv on K-1 folds
    rsf_stack.fit(X_train_selected, y_train[train_idx])
    print("RSF model trained")

    # compute the RSF risk score on the train set
    rsf_train_pred = rsf_stack.predict(X_train_selected).flatten()
    # print(rsf_train_pred.shape)
    rsf_selected_score = concordance_index_ipcw(y_train_org[train_idx], y_train_org[train_idx], rsf_train_pred, tau=7)[0]
    print(f"RSF IPCW C-index on train: {rsf_selected_score:.2f}")

    # deepsurv takes tensors as inputs
    ######################
    # Convert data to tensors
    X_train_t = torch.tensor(X_train_selected.values, dtype=torch.float32).to(device)
    X_val_t = torch.tensor(X_val_selected.values, dtype=torch.float32).to(device)

    # Convert target variables into PyTorch tensors (Manual conversion)
    y_train_selected_t = (
        torch.tensor(y_train['OS_YEARS'][train_idx].copy(), dtype=torch.float64).to(device),  # Time to event
        torch.tensor(y_train["OS_STATUS"][train_idx].copy(), dtype=torch.int64).to(device)    # Event occurred (1=event, 0=censored)
    )

    y_val_selected_t = (
        torch.tensor(y_train["OS_YEARS"][val_idx].copy(), dtype=torch.float64).to(device),  # Time to event
        torch.tensor(y_train["OS_STATUS"][val_idx].copy(), dtype=torch.int64).to(device)    # Event occurred (1=event, 0=censored)
    )
    #########################

    deepsurv_stack.fit(X_train_t, y_train_selected_t, batch_size=128, epochs=200, verbose=False)
    print("DeepSurv model trained")

    # compute risk scores
    deepsurv_train_pred = deepsurv_stack.predict(X_train_t).cpu().detach().numpy().reshape(-1)
    deepsurv_selected_score = concordance_index_ipcw(y_train_org[train_idx], y_train_org[train_idx], deepsurv_train_pred, tau=7)[0]
    print(f"DeepSurv IPCW C-index on train: {deepsurv_selected_score:.2f}")


    #################
    # Predict on validation fold
    train_rsf_oof[val_idx] = rsf_stack.predict(X_val_selected).flatten()
    train_deepsurv_oof[val_idx] = deepsurv_stack.predict(X_val_t).cpu().detach().numpy().reshape(-1)

    # compute the RSF risk score on the validation set
    rsf_val_pred = rsf_stack.predict(X_val_selected).flatten()
    rsf_selected_score = concordance_index_ipcw(y_train_org[val_idx], y_train_org[val_idx], rsf_val_pred, tau=7)[0]
    print(f"RSF IPCW C-index on validation: {rsf_selected_score:.2f}")

    # compute the DeepSurv risk score on the validation set
    deepsurv_val_pred = deepsurv_stack.predict(X_val_t).cpu().detach().numpy().reshape(-1)
    deepsurv_selected_score = concordance_index_ipcw(y_train_org[val_idx], y_train_org[val_idx], deepsurv_val_pred, tau=7)[0]
    print(f"DeepSurv IPCW C-index on validation: {deepsurv_selected_score:.2f}")


In [ ]:
# Compute the predictions and the score on the X_test and y_test data
rsf_test_pred = rsf_stack.predict(X_test_znorm).flatten()
rsf_test_score = concordance_index_ipcw(y_train_org, y_test, rsf_test_pred, tau=7)[0]
print(f"RSF IPCW C-index on TEST: {rsf_test_score:.2f}")

# compute the DeepSurv risk score on the "test" set
deepsurv_test_pred = deepsurv_stack.predict(X_test_t).cpu().detach().numpy().reshape(-1)
deepsurv_test_score = concordance_index_ipcw(y_train_org, y_test, deepsurv_test_pred, tau=7)[0]
print(f"DeepSurv IPCW C-index on TEST: {deepsurv_test_score:.2f}")

In [ ]:
# Stack out-of-fold predictions
X_meta_train = np.column_stack((train_rsf_oof, train_deepsurv_oof))
# print(X_meta_train.shape)

# Train meta-learner
# meta_learner = LinearRegression()
# meta_learner.fit(X_meta_train, y_train["OS_YEARS"])

# Try a XGBoost Regressor :
import xgboost as xgb
meta_learner = xgb.XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.005, objective='reg:squarederror')
meta_learner.fit(X_meta_train, y_train["OS_YEARS"])

# Final test predictions
test_rsf_preds = rsf_stack.predict(X_test_znorm).flatten()

# to tensor
X_test_t = torch.tensor(X_test_znorm.values, dtype=torch.float32).to(device)
test_deepsurv_preds = deepsurv_stack.predict(X_test_t).cpu().detach().numpy().reshape(-1)
# print(test_deepsurv_preds.shape)
# print(X_test_znorm.shape)
# print(y_test.shape)

X_meta_test = np.column_stack((test_rsf_preds, test_deepsurv_preds))

# Predict final risk scores
final_predictions = -meta_learner.predict(X_meta_test)

meta_test_cindex = concordance_index_ipcw(y_train_org, y_test, final_predictions, tau=7)[0]
print(f"Meta-Learner IPCW C-index on test: {meta_test_cindex:.2f}")

In [ ]:
# # Trying to fit an isotonic regression
# from sklearn.isotonic import IsotonicRegression
# iso_reg = IsotonicRegression()

In [ ]:
################ Now predict on the platform data
# df_eval.head()

# Preprocess df_eval (same transformation as training)
# apply imputation
df_eval[features] = imputer.transform(df_eval[features])

# Z normalize the df_eval data
X_eval_znorm = pd.DataFrame(data_scaler.transform(df_eval[features]), columns=features, index=df_eval.index)

# transform to tensor
X_eval_t = torch.tensor(X_eval_znorm.values, dtype=torch.float32).to(device)

In [ ]:
rsf_platform_preds = rsf_stack.predict(X_eval_znorm).flatten()

# rsf repetition count : 
tmp_series = pd.Series(rsf_platform_preds)
print(tmp_series.value_counts())


platform_deepsurv_preds = deepsurv_stack.predict(X_eval_t).cpu().detach().numpy().reshape(-1)

X_platform_meta = np.column_stack((rsf_platform_preds, platform_deepsurv_preds))
platform_final_preds = -meta_learner.predict(X_platform_meta)

platform_submission = pd.Series(platform_final_preds, index=df_eval['ID'], name='risk_score')
platform_submission


# check if there are repeating values in the submission risk scores
print(platform_submission.value_counts())
# platform_submission.to_csv(f"./output/submission_{now_str()}_meta_xgregressor.csv", header=True)
# print("Submission file saved as 'submission meta learner regression.csv'")

In [ ]:
corr = np.corrcoef(train_rsf_oof, train_deepsurv_oof)[0, 1]
print(f"Correlation between RSF and DeepSurv: {corr:.2f}")

#### Cross Validation on DeepSurv

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
c_indices = []

for train_idx, val_idx in kf.split(X_train_znorm):
    X_tr, X_val = X_train_znorm.iloc[train_idx], X_train_znorm.iloc[val_idx]
    y_tr = y_train[train_idx]
    y_val = y_train[val_idx]
    
    X_tr_t = torch.tensor(X_tr.values, dtype=torch.float32).to(device)
    X_val_t = torch.tensor(X_val.values, dtype=torch.float32).to(device)

    y_tr_t = (
        torch.tensor(y_tr["OS_YEARS"].copy(), dtype=torch.float32).to(device),
        torch.tensor(y_tr["OS_STATUS"].copy(), dtype=torch.int64).to(device)
    )

    # TODO : vérifier les dtype accross diff methods float_cmb
    
    # TODO : fix this 
    net = MLPVanilla(
        in_features=X_tr.shape[1], num_nodes=[64, 32, 16],
        out_features=1, batch_norm=True, dropout=0.2, output_bias=False
    ).to(device)
    
    model = CoxPH(net, optimizer=torch.optim.Adam)
    model.optimizer.set_lr(0.0001)
    # # TODO : LR et weight decay??
    # model.optimizer.set_lr(0.00005)  # 0.0001
    model.optimizer.weight_decay = 1e-4  # L2 regularization
    
    model.fit(X_tr_t, y_tr_t, batch_size=128, epochs=200, verbose=False)
    
    val_scores = model.predict(X_val_t).cpu().detach().numpy().reshape(-1)

    c_index = concordance_index_ipcw(y_tr, y_val, val_scores, tau=7)[0]
    print(f"Validation C-index: {c_index}")
    c_indices.append(c_index)

print(f"Cross-Validation Mean IPCW C-index: {np.mean(c_indices)}")


In [ ]:
# After cross validation, we can train the model on the entire training data and make predictions on the test data
# question : training all the models on the entire training data? rather than just the train data?


#### Predicting with the DeepSurv model

In [ ]:
# Preprocess df_eval (same transformation as training)
# apply imputation
df_eval[features] = imputer.transform(df_eval[features])
# X_eval = imputer.transform(df_eval[features])     ---- mal noté

# Z normalize the df_eval data
X_eval_znorm = pd.DataFrame(data_scaler.transform(df_eval[features]), columns=features, index=df_eval.index)

# transform to tensor
X_eval_t = torch.tensor(X_eval_znorm.values, dtype=torch.float32).to(device)

In [ ]:
# Predict risk scores using DeepSurv
eval_deep_risk_scores = model.predict(X_eval_t).cpu().detach().numpy().reshape(-1)  # Convert to 1D

# Create submission file
deep_test_submission = pd.Series(eval_deep_risk_scores, index=df_eval['ID'], name='risk_score')

deep_test_submission

# Save as CSV
deep_test_submission.to_csv(f"./output/submission_{now_str()}_deepsurv.csv", header=True)
print("Submission file saved as 'submission_date_deepsurv.csv'")


In [ ]:
# TODO : maybe too many epochs genre 200 too much?

#### Training a meta model on RSF + DeepSurv

In [ ]:
from sklearn.linear_model import LinearRegression
from sksurv.metrics import concordance_index_ipcw

# Get predictions from RSF and DeepSurv models
rsf_train_preds = rsf.predict(X_train_znorm)
rsf_test_preds = rsf.predict(X_test_znorm)
deepsurv_train_preds = model.predict(X_train_t).cpu().detach().numpy().reshape(-1)
deepsurv_test_preds = model.predict(X_test_t).cpu().detach().numpy().reshape(-1)

# Stack predictions as features for the meta-learner
stacked_train_preds = pd.DataFrame({
    'rsf': rsf_train_preds,
    'deepsurv': deepsurv_train_preds
})

stacked_test_preds = pd.DataFrame({
    'rsf': rsf_test_preds,
    'deepsurv': deepsurv_test_preds
})

# Train a linear regression model as the meta-learner
meta_learner = LinearRegression()
meta_learner.fit(stacked_train_preds, y_train['OS_YEARS'])

# Get meta-learner predictions
meta_train_preds = -meta_learner.predict(stacked_train_preds)
meta_test_preds = -meta_learner.predict(stacked_test_preds)

# Evaluate the meta-learner using IPCW Concordance Index
meta_train_cindex = concordance_index_ipcw(y_train, y_train, meta_train_preds, tau=7)[0]
meta_test_cindex = concordance_index_ipcw(y_train, y_test, meta_test_preds, tau=7)[0]

print(f"Meta-Learner IPCW C-index on train: {meta_train_cindex:.2f}")
print(f"Meta-Learner IPCW C-index on test: {meta_test_cindex:.2f}")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sksurv.metrics import concordance_index_ipcw

# Get predictions from RSF and DeepSurv models
rsf_train_preds = rsf.predict(X_train_znorm)
rsf_test_preds = rsf.predict(X_test_znorm)
deepsurv_train_preds = model.predict(X_train_t).cpu().detach().numpy().reshape(-1)
deepsurv_test_preds = model.predict(X_test_t).cpu().detach().numpy().reshape(-1)

# # Stack predictions as features for the meta-learner

# Z-normalize the RSF and DeepSurv predictions individually
rsf_scaler = StandardScaler()
deepsurv_scaler = StandardScaler()

rsf_train_preds_znorm = rsf_scaler.fit_transform(rsf_train_preds.reshape(-1, 1)).flatten()
rsf_test_preds_znorm = rsf_scaler.transform(rsf_test_preds.reshape(-1, 1)).flatten()

deepsurv_train_preds_znorm = deepsurv_scaler.fit_transform(deepsurv_train_preds.reshape(-1, 1)).flatten()
deepsurv_test_preds_znorm = deepsurv_scaler.transform(deepsurv_test_preds.reshape(-1, 1)).flatten()

# Stack normalized predictions as features for the meta-learner
stacked_train_preds = pd.DataFrame({
    'rsf': rsf_train_preds_znorm,
    'deepsurv': deepsurv_train_preds_znorm
})

stacked_test_preds = pd.DataFrame({
    'rsf': rsf_test_preds_znorm,
    'deepsurv': deepsurv_test_preds_znorm
})

# Train a Gradient Boosting Regressor as the meta-learner
meta_learner = GradientBoostingRegressor(n_estimators=100, learning_rate=0.005, min_samples_split=10,  max_depth=3, random_state=42, validation_fraction=0.3)
meta_learner.fit(stacked_train_preds, y_train['OS_YEARS'])

# Get meta-learner predictions
meta_train_preds = -meta_learner.predict(stacked_train_preds)
meta_test_preds = -meta_learner.predict(stacked_test_preds)

# Evaluate the meta-learner using IPCW Concordance Index
meta_train_cindex = concordance_index_ipcw(y_train, y_train, meta_train_preds, tau=7)[0]
meta_test_cindex = concordance_index_ipcw(y_train, y_test, meta_test_preds, tau=7)[0]

print(f"Meta-Learner IPCW C-index on train: {meta_train_cindex:.2f}")
print(f"Meta-Learner IPCW C-index on test: {meta_test_cindex:.2f}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Grid search on the meta-learner

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.001, 0.005, 0.01],
    'n_estimators': [100, 200, 300],
    'validation_fraction': [0.3],
    'random_state': [42],
    'min_samples_split': [2, 5, 10],
}

# Initialize the XGBoost model
xgb_meta_model = GradientBoostingRegressor()            # TODO : vs XGBRegressor

# Initialize GridSearchCV with the XGBoost model and parameter grid
grid_search = GridSearchCV(estimator=xgb_meta_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the grid search to the stacked training data
grid_search.fit(stacked_train_preds, y_train['OS_YEARS'])

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters found: {best_params}")
print(f"Best score found: {best_score}")

# Train the XGBoost model with the best parameters
best_xgb_meta_model = xgb.XGBRegressor(**best_params)
best_xgb_meta_model.fit(stacked_train_preds, y_train['OS_YEARS'])

# Get meta-learner predictions
meta_train_preds = -best_xgb_meta_model.predict(stacked_train_preds)
meta_test_preds = -best_xgb_meta_model.predict(stacked_test_preds)

# Evaluate the meta-learner using IPCW Concordance Index
meta_train_cindex = concordance_index_ipcw(y_train, y_train, meta_train_preds, tau=7)[0]
meta_test_cindex = concordance_index_ipcw(y_train, y_test, meta_test_preds, tau=7)[0]

print(f"Meta-Learner IPCW C-index on train: {meta_train_cindex:.2f}")
print(f"Meta-Learner IPCW C-index on test: {meta_test_cindex:.2f}")

#### Inference time

In [ ]:
# Predict on test set :
# Get predictions from RSF and DeepSurv models
rsf_test_preds = rsf.predict(X_eval_znorm)
deepsurv_test_preds = model.predict(X_eval_t).cpu().detach().numpy().reshape(-1)

# Z-normalize the RSF and DeepSurv predictions individually
rsf_test_preds_znorm = rsf_scaler.transform(rsf_test_preds.reshape(-1, 1)).flatten()
deepsurv_test_preds_znorm = deepsurv_scaler.transform(deepsurv_test_preds.reshape(-1, 1)).flatten()

# Stack normalized predictions as features for the meta-learner
stacked_test_preds = pd.DataFrame({
    'rsf': rsf_test_preds_znorm,
    'deepsurv': deepsurv_test_preds_znorm
})

# Get meta-learner predictions
meta_test_preds = -meta_learner.predict(stacked_test_preds)

# Create submission file
meta_test_submission = pd.Series(meta_test_preds, index=df_eval['ID'], name='risk_score')
meta_test_submission


In [ ]:
# save the submission
meta_test_submission.to_csv(f"./output/submission_{now_str()}_meta_learner_linear.csv", header=True)
print("Submission file saved as 'submission_date_meta_learner.csv'")

#### Trying the DeepHit model

In [ ]:
# from pycox.models import DeepHitSingle

# net = MLPVanilla(
#     in_features=X_train.shape[1], num_nodes=[64, 32], out_features=1, batch_norm=True, dropout=0.2
# ).to(device)

# model = DeepHitSingle(net, optimizer=torch.optim.Adam)
# model.fit(X_train_t, y_train_t, batch_size=128, epochs=200, verbose=True)

# train_hit_risk_scores = model.predict(X_train_t).cpu().detach().numpy().reshape(-1)

# # compute train and test c-index
# hit_train_cindex = concordance_index_censored(y_train["OS_STATUS"], y_train["OS_YEARS"], train_hit_risk_scores)[0]
# print(f"DeepSurv IPCW C-index on train: {deep_train_cindex}")